# <font color=hotpink> Review Materials </font> 

## <font color=gold> Recommendation systems </font>

## Make pivot table from dataframe

## Show data in browser

In [5]:
import pandas as pd
import numpy as np
import os
import webbrowser

# Read the dataset into a data table using Pandas
df = pd.read_csv("movie_ratings_data_set.csv")

# Convert the running list of user ratings into a matrix using the 'pivot table' function
ratings_df = pd.pivot_table(df,index= 'user_id', columns='movie_id')

# Create a web page view of the data for easy viewing
html = ratings_df.to_html(na_rep="")

# Save the html to a temporary file
with open("review_matrix.html", "w") as f:
    f.write(html)

# Open the web page in our web browser
full_filename = os.path.abspath("review_matrix.html")
webbrowser.open("file://{}".format(full_filename))

True

***
***.as_matrix():** make sure that the data is passed as a numpy matrix

***
Save features and predicted ratings to files for later use
<br><br>This helps to save large matrix U faster.
<br><br><font color=green>pickle.dump(U, open("user_features.dat", "wb"))

***

#### <font color=green>import pickle 
import pandas as pd </font>
<br>
Load prediction rules from data files
<br>
<font color=green>means = pickle.load(open("means.dat", "rb"))</font>
<br>
The pickle feature is useful in running the recommendation on pre-saved model data .

***
**MovieLens DataSet**

### Main Idea:
First, convert the data into meaningful table using pd.read_csv and pd.pivot_table. 
<br> Then instead of finding features of each movie by yourself, do low_rank_matrix_factorization to automatically come up with features of movies. In other words, data = U * M. 
<br> Factor a ratings array into two latent feature arrays (user features and product features) as in the following:

In [7]:
def low_rank_matrix_factorization(ratings, mask=None, num_features=15, regularization_amount=0.01):
    """
    Factor a ratings array into two latent feature arrays (user features and product features)

    :param ratings: Matrix with user ratings to factor
    :param mask: A binary mask of which ratings are present in the ratings array to factor
    :param num_features: Number of latent features to generate for users and products
    :param regularization_amount: How much regularization to apply
    :return: (P, Q) - the factored latent feature arrays
    """

This can be done by calculating U and M repeatedly using

In [8]:
from scipy.optimize import fmin_cg

which minimizes the error using gradient calculation. 

Then:
<br>
predicted_ratings = np.matmul(U, M)

Then, for a certain user_ID, one can sort the predicted_ratings and suggest the top five rating for the user. 

***
If a user doesn't have reviews so far, we can either:
<br>
Not suggest anything, or suggest the top rated movies, or suggest related movies.

Related movies are the ones with the most similar M matrix. Also, don't forget regularization.